# Imports

In [11]:
from mtgsdk import Card as crd
from IPython.display import display, Image
from PIL import Image
import requests
import io
import pickle as pkl
import pandas as pd
import numpy as np
import urllib.request

# Getting the Database

In [12]:
cards = crd.all()
cDict = [c.__dict__ for c in cards]
df = pd.DataFrame(cDict)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35983 entries, 0 to 35982
Data columns (total 39 columns):
artist            35983 non-null object
border            22 non-null object
cmc               35983 non-null float64
color_identity    31897 non-null object
colors            28223 non-null object
flavor            21356 non-null object
foreign_names     23871 non-null object
hand              116 non-null float64
id                35983 non-null object
image_url         33753 non-null object
layout            35983 non-null object
legalities        35171 non-null object
life              116 non-null float64
loyalty           203 non-null float64
mana_cost         31671 non-null object
multiverse_id     33753 non-null float64
name              35983 non-null object
names             475 non-null object
number            29781 non-null object
original_text     33298 non-null object
original_type     34034 non-null object
power             16760 non-null object
printings        

In [13]:

df[["name", "image_url"]].head(10)
dftemp = df[['name', 'image_url']].dropna()
dftemp.head(10)

,name,image_url
0,Adorable Kitten,http://gatherer.wizards.com/Handlers/Image.ash...
1,Aerial Toastmaster,http://gatherer.wizards.com/Handlers/Image.ash...
2,Amateur Auteur,http://gatherer.wizards.com/Handlers/Image.ash...
3,Amateur Auteur,http://gatherer.wizards.com/Handlers/Image.ash...
4,Amateur Auteur,http://gatherer.wizards.com/Handlers/Image.ash...
5,Amateur Auteur,http://gatherer.wizards.com/Handlers/Image.ash...
6,By Gnome Means,http://gatherer.wizards.com/Handlers/Image.ash...
7,Chivalrous Chevalier,http://gatherer.wizards.com/Handlers/Image.ash...
8,Do-It-Yourself Seraph,http://gatherer.wizards.com/Handlers/Image.ash...
9,Gimme Five,http://gatherer.wizards.com/Handlers/Image.ash...


In [14]:
import urllib.request

urllib.request.urlretrieve(dftemp["image_url"][0], "local-filename.jpg")

('local-filename.jpg', <http.client.HTTPMessage at 0xa0bce48>)

In [19]:
dftemp["image_url"][1]

'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=439391&type=card'

In [25]:
import re
import requests
from bs4 import BeautifulSoup

site = dftemp["image_url"][1]

response = requests.get(site)

soup = BeautifulSoup(response.text, 'html.parser')
img_tags = soup.find_all('img')

urls = [img['src'] for img in img_tags]

for url in urls:
    filename = re.search(r'/([\w_-]+[.](jpg|gif|png))$', url)
    print(filename)
    with open(filename.group(1), 'wb') as f:
        if 'http' not in url:
            # sometimes an image source can be relative 
            # if it is provide the base url which also happens 
            # to be the site variable atm. 
            url = '{}{}'.format(site, url)
        response = requests.get(url)
        f.write(response.content)